In [2]:
pip install pymysql # for replication in new test environments

  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)
Note: you may need to restart the kernel to use updated packages.


## Run the block of code and after asked, share your news title, author, article

In [6]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import itertools
import pickle
import pymysql
from sqlalchemy import create_engine
from datetime import datetime
# load the model from disk
clf = pickle.load(open('randomForest_model.sav', 'rb'))
Tfidmodel = pickle.load(open('Tfidmodel.sav', 'rb'))


tempAuthor = input("Please enter the name of author:\n")
tempTitle = input("Please enter the artical title:\n")
tempText = input("Please enter the artical text:\n")
tempList = []
tempList.append(tempAuthor+' '+tempTitle+' '+tempText)

# Tfidf transformation to inputdata
tempX = Tfidmodel.transform(tempList)
tempY = clf.predict(tempX)
tempYprob = clf.predict_proba(tempX)

db = pymysql.connect(host='fakenews3.ch8lvxaws3z2.us-east-1.rds.amazonaws.com',user='admin',password='!QAZ2wsx',database ='fakenews',charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor)
cursor = db.cursor()
cursor.connection.commit()

now=datetime.now().strftime("%Y/%m/%d %H:%M:%S")

if tempY == '1':
    print ("fake news!!")
else :
    print ("good news!")

try:
    sql = '''insert into fakenews(id, author, title, text, prediction,created_time) values(NULL,'%s','%s','%s', '%s','%s')''' % (tempAuthor, tempTitle, tempText,str(tempY[0]),now)
    cursor.execute(sql)
    db.commit()
    print( "Data Added to the Table Successfully!")
except Exception as e:
    print(e)

Please enter the name of author:
April Hunt
Please enter the artical title:
Emory senior Ahmed Aljohani awarded prestigious Rhodes Scholarship
Please enter the artical text:
Emory University senior Ahmed Aljohani has been awarded the prestigious 2022 Rhodes Scholarship, which provides all expenses for two or three years of study at the University of Oxford in England.  Aljohani, a biology major in Emory College of Arts and Sciences and a native of Saudi Arabia, is among an international group of Rhodes Scholars who join more than 100 scholars selected to begin study next fall. He is Emory’s 21st recipient of the award and the university’s first selected from Saudi Arabia.  “I never imagined such an honor. I don’t think I would have been accepted to the Rhodes without being so well rounded,” Aljohani says. “I owe that to Emory, for being a place to explore yourself and develop skills as a full person. That is essential in this world.”  Aljohani exemplifies the traits of a Rhodes Scholar

/var/folders/50/y6x0nd9x4z132jyt97tw796w0000gp/T/ipykernel_23451/3753924222.py:32: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if tempY == '1':


## Decoding the results.
For verfied news, the code will return "good news !"

For fake news, the code will return "fake news !!!"